<a href="https://colab.research.google.com/github/sanjay2097/NYC-Taxi-Trip-Time-Prediction/blob/main/NYC_Taxi_Time_Prediction_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <b> Data Description </b>

### The dataset is based on the 2016 NYC Yellow Cab trip record data made available in Big Query on Google Cloud Platform. The data was originally published by the NYC Taxi and Limousine Commission (TLC). The data was sampled and cleaned for the purposes of this project. Based on individual trip attributes, you should predict the duration of each trip in the test set.

### <b>NYC Taxi Data.csv</b> - the training set (contains 1458644 trip records)


### Data fields
* #### id - a unique identifier for each trip
* #### vendor_id - a code indicating the provider associated with the trip record
* #### pickup_datetime - date and time when the meter was engaged
* #### dropoff_datetime - date and time when the meter was disengaged
* #### passenger_count - the number of passengers in the vehicle (driver entered value)
* #### pickup_longitude - the longitude where the meter was engaged
* #### pickup_latitude - the latitude where the meter was engaged
* #### dropoff_longitude - the longitude where the meter was disengaged
* #### dropoff_latitude - the latitude where the meter was disengaged
* #### store_and_fwd_flag - This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server - Y=store and forward; N=not a store and forward trip
* #### trip_duration - duration of the trip in seconds

## **STEPS** 
### 1. Data Analysis
### 2. Feature Engineering
### 3. Feature Selection
### 4. Model Building
### 5. Model Validation & Selection


In [ ]:
import numpy as np
import pandas as pd
from numpy import math
from datetime import datetime

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from lightgbm import LGBMRegressor

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

import seaborn as sns
import folium
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

### DATA ANALYSIS

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Loading Data
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NYC Taxi Data.csv')

In [ ]:
dataset.head()

In [ ]:
dataset.shape

(1458644, 11)

There are approx 1.46 million records in our dataset.

In [ ]:
dataset.describe()

From preliminary analysis using describe function we can see that there are anomalous values in passenger count and trip_duration that needs to be addressed later.

In [ ]:
# Checking null values
dataset.isnull().sum()

In [ ]:
# Checking duplicated values
dataset.duplicated().sum()

0

*There are no Null values and duplicated values in given dataset.*

In [ ]:
# Copying data to new dataframe for further analysis
df = dataset.copy()

In [ ]:
# Lets look at the distribution plot of the features
pos = 1
fig = plt.figure(figsize=(18,26))
for i in df.describe().columns:
    ax = fig.add_subplot(6,2,pos)
    pos = pos + 1
    sns.distplot(df[i],ax=ax)

Inferences from distribution plot :

1.There are two major vendors in NYC.

2.Passenger count 1 has max distribution.

3.Distribution of trip duration is highly skewed.

### Analysis of independent variables

#### Vendor ID

In [ ]:
df['vendor_id'].value_counts().plot(kind='bar')
plt.ylabel('Count')
plt.xlabel('Vendor ID')

*Both the vendors seems to have almost equal market share. But Vendor 2 is evidently more famous among the population as per the above graph.*

#### Datetime

In [ ]:
# Converting datetime datatype from object
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'])

In [ ]:
# Adding new features month , day and hour from datetime
df['hour'] = df['pickup_datetime'].dt.hour 
df['day'] = df.pickup_datetime.dt.day_name()
df['month'] = df.pickup_datetime.dt.month_name()

In [ ]:
# Analyzing month 
data=df['month']
plt.figure(figsize=(8,5))
sns.countplot(data, palette='rainbow')
plt.show()

All the months are closely distributed with March being highest and January lowest.

In [ ]:
# Analyzing day
data=df['day']
plt.figure(figsize=(8,5))
sns.countplot(data, palette='rainbow')
plt.show()

We can see that Friday has the largest count of trips in dataset and Monday lowest.

In [ ]:
# Analyzing hour
data=df['hour']
plt.figure(figsize=(8,5))
sns.countplot(data, palette='rainbow')
plt.show()

### Passanger Count

Between 7 am to 3 pm the trip distributions are close to each other but increases from 5 pm to 10 pm and starts decreasing till 5 am.

In [ ]:
# Analyzing passanger count
data=df['passenger_count']
plt.figure(figsize=(8,5))
sns.countplot(data, palette='rainbow')
plt.show()

The passenger_count variable has a minimum value of 0 passengers. These observations are most likely errors and will need to be removed from the dataset.

According to the NYC Taxi & Limousine Commission, the maximum number of people allowed in a yellow taxicab, by law, is 5 passengers and one child .The observations more than 6 are likely an error and will also need to be removed from the dataset.

In [ ]:
# Removing passenger count more than 6 
df = df[(df['passenger_count']>0) & (df['passenger_count']<=6)]

###store_and_fwd_flag

In [ ]:
# analyzing trip data storing flag column
df['store_and_fwd_flag'].value_counts()

Most of the trip records were not stored in vehicle memory before forwarding to the vendor because the vehicle did not have a direct connection to the server.

#### Longitude and Latitude

Looking into it, the borders of NY City coordinates comes out to be:

longitude = (-74.03, -73.77)  ,
latitude = (40.63, 40.85)


Any coordinates outside will be outliers.

In [ ]:
# Max and min values of lat and long in pickup and dropoff location
print(np.min(df['pickup_longitude']), np.min(df['pickup_latitude']))
print(np.max(df['pickup_longitude']), np.max(df['pickup_latitude']))

print(np.min(df['dropoff_longitude']), np.min(df['dropoff_latitude']))
print(np.max(df['dropoff_longitude']), np.max(df['dropoff_latitude']))

In [ ]:
# Removing outlier coordinates
west, south, east, north = -74.03, 40.63, -73.77, 40.85

df = df[(df.pickup_latitude> south) & (df.pickup_latitude < north)]
df = df[(df.dropoff_latitude> south) & (df.dropoff_latitude < north)]
df = df[(df.pickup_longitude> west) & (df.pickup_longitude < east)]
df = df[(df.dropoff_longitude> west) & (df.dropoff_longitude < east)]

In [ ]:
# Visualization of coordinates
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True,figsize=(18,10))

df.plot(kind='scatter', x='pickup_longitude', y='pickup_latitude',
                color='yellow', 
                s=.02, alpha=.6, subplots=True, ax=ax1)
ax1.set_title("Pickups")
ax1.set_facecolor('black')

df.plot(kind='scatter', x='dropoff_longitude', y='dropoff_latitude',
                color='yellow', 
                s=.02, alpha=.6, subplots=True, ax=ax2)
ax2.set_title("Dropoffs")
ax2.set_facecolor('black') 

In [ ]:
# Finding total diatance covered in each trip by making get_distance function
from math import sin, cos, sqrt, atan2, radians

def get_distance(lon_1, lon_2, lat_1, lat_2):

    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat_1)
    lon1 = radians(lon_1)
    lat2 = radians(lat_2)
    lon2 = radians(lon_2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance

In [ ]:
# Applying get_distance function to claculate each trip distance
df["distance"] = df.apply(lambda x: get_distance(x["pickup_longitude"],x["dropoff_longitude"],x["pickup_latitude"],x["dropoff_latitude"]),axis=1)

#### Distance

In [ ]:
df.distance.describe()

count    1.438573e+06
mean     3.292866e+00
std      3.662317e+00
min      0.000000e+00
25%      1.224953e+00
50%      2.068546e+00
75%      3.767414e+00
max      2.720017e+01
Name: distance, dtype: float64

We can see from describe function that there are trips where trip distance is zero which is not possible we need to drop these.

In [ ]:
# Boxplot of distance
plt.figure(figsize = (12,5))
sns.boxplot(df.distance)
plt.show()

During previous analysis of longitude and latitude columns we have limited all the trips within NY city only , hence all the outliers are actually part of extreme values and we have no further reason to remove them.

In [ ]:
# Total outlier values where value is 0
len(df[df.distance==0])

71929

In [ ]:
# Removing outliers 
df = df[df.distance>0]

In [ ]:
# Plotting distribution of trip distribution
from scipy import stats
from scipy.stats import norm, skew

plt.rcParams["figure.figsize"] = (12,6)
sns.distplot(df['distance'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(df['distance'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

# Plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('Duration distribution')
plt.show()


### Bivariate Analysis

In [ ]:
# Analysis between pickup hour and trip duration
group1 = df.groupby('hour').trip_duration.median()
plt.figure(figsize = (12,6))
sns.pointplot(x=group1.index, y=group1.values)
plt.ylabel('Average Trip Duration')
plt.xlabel('Pickup Hour')
plt.show()

There is sharp increase in avg trip duration from 6 am onwards till 10 am ,  most working people leave at these hours.

In [ ]:
# Analysis between weekday and trip duration
group1 = df.groupby('day').trip_duration.median()
plt.figure(figsize = (12,6))
sns.pointplot(x=group1.index, y=group1.values)
plt.ylabel('Average Trip Duration')
plt.xlabel('Weekday')
plt.show()

The avg trip duration on sunday is slightly lower than other days , it could be because of weekend and less number of people going out.

In [ ]:
# Analysis between pickup hour and trip distance
group1 = df.groupby('day').distance.median()
plt.figure(figsize = (12,6))
sns.pointplot(x=group1.index, y=group1.values)
plt.ylabel('Average Trip Distance')
plt.xlabel('Weekday')
plt.show()

On sunday the trips cover more avg distance than other days , it could be because more travelling to and from airports or preople going out to meet friends and family from farther distances.

In [ ]:
# Day-wise comparison of traffic
n = sns.FacetGrid(df, col='day')
n.map(plt.hist, 'hour')
plt.show()

From above we can see that Weekends have high late night trip distribution as compared to other days as people tend to stay out late.

#### Analyzing target variable

#### Trip Duration

In [ ]:
df.trip_duration.describe()

In [ ]:
# boxplot trip duration
plt.figure(figsize = (12,5))
sns.boxplot(df.trip_duration)
plt.show()

In [ ]:
# Total outlier values outside 3 standard deviation
len(df[df.trip_duration>df.trip_duration.quantile(0.99)])

14326

In [ ]:
# Removing outliers 
df = df[df.trip_duration<df.trip_duration.quantile(0.99)]

In [ ]:
# Plotting trip distribution
from scipy import stats
from scipy.stats import norm, skew

plt.rcParams["figure.figsize"] = (12,6)
sns.distplot(df['trip_duration'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(df['trip_duration'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

# Plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('Duration distribution')
plt.show()


### Feature Selection

In [ ]:
# Copying into new dataframe
new_df=df.copy()

In [ ]:
new_df.info()

In [ ]:
# Removing unnecessary features which are of no use
new_df.drop(['id','pickup_datetime','dropoff_datetime','store_and_fwd_flag'], 
                axis = 1, inplace = True)

In [ ]:
# Removing duplicates
new_df.drop_duplicates(keep=False, inplace=True)

In [ ]:
# Visualising the correlation between attributes
corr = new_df.corr(method='kendall')
plt.figure(figsize=(16,10))
plt.title("Correlation Between Different Variables\n")
sns.heatmap(corr, annot=True)
plt.show()

There are no high correlations in our dataset.

In [ ]:
# Encoding Categorical Data
df1 = pd.get_dummies(new_df, columns=['vendor_id','passenger_count','hour','month','day'], drop_first=True)

In [ ]:
df1.info()

In [ ]:
# Preliminary analysis using stats OLS
import statsmodels.api as sm
x =   df1.loc[:, df1.columns != 'trip_duration']
Y = df1['trip_duration']

In [ ]:
x = sm.add_constant(x)
model= sm.OLS(Y, x).fit()
model.summary()